In [1]:
import cv2
import numpy as np
face_det = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#Activating my webcam
cap = cv2.VideoCapture(0)  
#defining a function to detect and extract cropped faces
def face_ext(image):
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    img = face_det.detectMultiScale(img_gray, 1.3, 5)
    
    if img is ():
        return None
    
    for (x,y,w,h) in img:
        crop_face = image[y:y+h, x:x+w]
    
    return crop_face
  
#Creating dataset for training model for face detection
i=0
Training_Data, Labels = [], []
while i<150:
    ret, photo = cap.read()
    if face_ext(photo) is not None:
        face = cv2.resize(face_ext(photo),(200, 200))
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        Training_Data.append(np.asarray(face, dtype=np.uint8))
        Labels.append(i)
        i += 1
        
       
        
        #displaying the dataset
        cv2.putText(face, str(i), (50, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)
        cv2.imshow('Gathering Training Data', face)
    else:
        print("No face found")
        pass
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()
Labels = np.asarray(Labels, dtype=np.int32)
print("Dataset Generated")

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-40f60cc4d5cd>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if img is ():


No face found
No face found
No face found
No face found
No face found
No face found
No face found
No face found
No face found
Dataset Generated


In [2]:
model  = cv2.face_LBPHFaceRecognizer.create() 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

Model trained sucessefully


In [3]:
import os
import boto3

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            disp_text = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, disp_text, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,100,100), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey AAKASH :)", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
            cv2.imshow('Face Recognition', image )
        
        else:
            cv2.putText(image, "Not sure who you are", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Detected", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,111,255), 2)
        cv2.putText(image, "Looking for a face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13:
        break
    
        
cap.release()
cv2.destroyAllWindows()

#Launching an ec2 instance and a 5GB instance volume if second face is recognized:
if confidence > 90:
    #for launching instance
    ec2 = boto3.resource('ec2')

# create a new EC2 instance
    instances = ec2.create_instances(
            ImageId='ami-0ad704c126371a549',
         MinCount=1,
         MaxCount=1
        ,
         InstanceType='t2.micro',
         KeyName='awspass'
 )
   
    print("Instance Launched successfully")
    


<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-cd0fbd7fa199>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Instance Launched successfully


In [4]:
os.system("aws ec2 create-volume  --availability-zone ap-south-1a  --size 5  --volume-type gp2")
print("An EBS volume of size 5gb has been created")

An EBS volume of size 5gb has been created


In [5]:
os.system("aws ec2 attach-volume --volume-id vol-0a0cc97581bd0e04a --instance-id i-0a300540d68fb7a0e --device /dev/xvdf")

0